In [ ]:
from fund_lens_etl.flows.extract_fec_contributions import extract_fec_contributions_flow

# Optional: Check Prefect is installed
import prefect

import logging

# Configure root logger to show everything
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Also set the specific service logger
logging.getLogger('fund_lens_etl.services.fec_service').setLevel(logging.INFO)

print(f"Prefect version: {prefect.__version__}")

In [ ]:
# Test extraction with limited results
# noinspection PyArgumentEqualDefault
stats = extract_fec_contributions_flow(
    state="MD",
    two_year_transaction_period=2024,
    max_results=500
)

print("\n=== Extraction Statistics ===")
print(f"State: {stats['state']}")
print(f"Cycle: {stats['cycle']}")
print(f"Contributions fetched: {stats['contributions_fetched']}")  # Changed
print(f"Contributions stored: {stats['contributions_stored']}")    # Changed
print(f"Raw filing ID: {stats['raw_filing_id']}")                  # Added
print(f"Duration: {stats['duration_seconds']:.2f} seconds")

In [ ]:
# Fetch MORE records to get some new ones beyond the first 500
# noinspection PyArgumentEqualDefault
stats = extract_fec_contributions_flow(
    state="MD",
    two_year_transaction_period=2024,
    max_results=1000  # Double the amount
)

print("\n=== Extraction Statistics ===")
print(f"State: {stats['state']}")
print(f"Cycle: {stats['cycle']}")
print(f"Contributions fetched: {stats['contributions_fetched']}")
print(f"Contributions stored: {stats['contributions_stored']}")
print(f"Raw filing ID: {stats['raw_filing_id']}")
print(f"Duration: {stats['duration_seconds']:.2f} seconds")

In [ ]:
# Test with 1500 records - should get 500 NEW ones
# noinspection PyArgumentEqualDefault
stats = extract_fec_contributions_flow(
    state="MD",
    two_year_transaction_period=2024,
    max_results=1500
)

print("\n=== Extraction Statistics ===")
print(f"State: {stats['state']}")
print(f"Cycle: {stats['cycle']}")
print(f"Contributions fetched: {stats['contributions_fetched']}")
print(f"Contributions stored: {stats['contributions_stored']}")  # Should be ~500
print(f"Raw filing ID: {stats['raw_filing_id']}")
print(f"Duration: {stats['duration_seconds']:.2f} seconds")

In [ ]:
# Try Virginia instead
stats = extract_fec_contributions_flow(
    state="VA",
    two_year_transaction_period=2024,
    max_results=1000
)

print(f"\nVirginia: Fetched {stats['contributions_fetched']}, Stored {stats['contributions_stored']}")

In [ ]:
# Test with 2000 records - this will have a NEW file hash
# noinspection PyArgumentEqualDefault
stats = extract_fec_contributions_flow(
    state="MD",
    two_year_transaction_period=2024,
    max_results=2000
)

print("\n=== Extraction Statistics ===")
print(f"State: {stats['state']}")
print(f"Cycle: {stats['cycle']}")
print(f"Contributions fetched: {stats['contributions_fetched']}")
print(f"Contributions stored: {stats['contributions_stored']}")  # Should be ~500
print(f"Raw filing ID: {stats['raw_filing_id']}")
print(f"Duration: {stats['duration_seconds']:.2f} seconds")

In [ ]:
from sqlalchemy import create_engine, text
from fund_lens_etl.config import get_database_url

engine = create_engine(get_database_url())

with engine.connect() as conn:
    # Check raw filings
    raw_count = conn.execute(text("SELECT COUNT(*) FROM raw_filings")).scalar()
    print(f"Raw filings: {raw_count}")

    # Check staging contributions
    staging_count = conn.execute(text("SELECT COUNT(*) FROM fec_contributions_staging")).scalar()
    print(f"Staging contributions: {staging_count}")

In [ ]:
from sqlalchemy import create_engine, text
from fund_lens_etl.config import get_database_url

engine = create_engine(get_database_url())

with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT
            MIN(transaction_dt) as earliest,
            MAX(transaction_dt) as latest,
            COUNT(*) as total
        FROM fec_contributions_staging
    """)).fetchone()

    print(f"Date range: {result[0]} to {result[1]}")
    print(f"Total records: {result[2]:,}")

    # Also check metadata
    metadata = conn.execute(text("""
        SELECT last_processed_date, records_extracted
        FROM extraction_metadata
        WHERE source = 'fec_api'
    """)).fetchone()

    if metadata:
        print(f"\nMetadata says:")
        print(f"  Last processed date: {metadata[0]}")
        print(f"  Records in last run: {metadata[1]:,}")

In [ ]:
from sqlalchemy import create_engine, text
from fund_lens_etl.config import get_database_url

engine = create_engine(get_database_url())

with engine.connect() as conn:
    # First, let's see what columns exist
    columns = conn.execute(text("""
        SELECT column_name
        FROM information_schema.columns
        WHERE table_name = 'fec_contributions_staging'
        ORDER BY ordinal_position
    """)).fetchall()

    print("Available columns:")
    for col in columns:
        print(f"  - {col[0]}")

    # Get basic count
    count = conn.execute(text("""
        SELECT COUNT(*)
        FROM fec_contributions_staging
    """)).scalar()

    print(f"\nTotal records: {count:,}")

In [ ]:
from sqlalchemy import create_engine, text
from fund_lens_etl.config import get_database_url

engine = create_engine(get_database_url())

with engine.connect() as conn:
    # Count by month
    result = conn.execute(text("""
        SELECT
            LEFT(transaction_dt::text, 6) as year_month,
            COUNT(*) as count
        FROM fec_contributions_staging
        GROUP BY LEFT(transaction_dt::text, 6)
        ORDER BY year_month
    """)).fetchall()

    print("Contributions by month:")
    for row in result:
        ym = row[0]
        year = ym[:4]
        month = ym[4:6]
        print(f"  {year}-{month}: {row[1]:,}")